Package installations

In [ ]:
!pip install datasets
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.1 MB/s eta 0:00:00


In [ ]:
!pip freeze > requirements.txt

In [ ]:
import torch
import torch.nn as nn
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


Define a LoRA classes for replacing dense layers. LoRA layers have rank decomposition matrices based on a provided dense layer, rank, and alpha. LoRA modules containing the A and B matrices are separate that way the LoRA modules can be stored or interchanged and a common model can be used.

In [ ]:
class LoraModule(nn.Module):
  def __init__(self, in_features, out_features, rank, alpha):
    super().__init__()
    self.scale = alpha / rank
    # Matrix A initialized with random gaussian distrubtion with shape (r, d)
    self.A = nn.Parameter(torch.randn(in_features, rank))
    # Matrix B initialized to zeros with shape (d, r)
    self.B = nn.Parameter(torch.zeros(rank, out_features))

  def forward(self, x):
    return (self.scale * (x @ self.A @ self.B))

class LoraLinear(nn.Module):
  def __init__(self, linear, rank, alpha):
    super().__init__()
    self.linear = linear
    self.lora = LoraModule(linear.in_features, linear.out_features, rank, alpha)

  def forward(self, x):
    return self.linear(x) + self.lora(x)

Helper function for applying lora layers to a given model. Supports RoBERTa and DeBERTa.

In [ ]:
def init_lora_layers(rank, alpha, model):
  modules = []

  # Replace dense layers of the model with LoRA
  if "roberta" in model.__class__.__name__.lower():
    for i, layer in enumerate(model.roberta.encoder.layer):
      s = layer.attention.self

      # Replace query linear layer with a lora layer
      if (isinstance(s.query, LoraLinear)):
        s.query.lora = LoraModule(s.query.linear.in_features, s.query.linear.out_features, rank, alpha)
      else:
        s.query = LoraLinear(s.query, rank, alpha)

      # Replace value linear layer with a lora layer
      if (isinstance(s.value, LoraLinear)):
        s.value.lora = LoraModule(s.value.linear.in_features, s.query.linear.out_features, rank, alpha)
      else:
        s.value = LoraLinear(s.value, rank, alpha)

      # Store the lora layers for later
      modules.append({'query': s.query.lora, 'value': s.value.lora})

  elif "deberta" in model.__class__.__name__.lower():
    for i, layer in enumerate(model.deberta.encoder.layer):
      s = layer.attention.self

      # Replace query linear layer with a lora layer
      if (isinstance(s.query, LoraLinear)):
        s.query_proj.lora = LoraModule(s.query_proj.linear.in_features, s.query.linear.out_features, rank, alpha)
      else:
        s.query_proj = LoraLinear(s.query_proj, rank, alpha)

      # Replace value linear layer with a lora layer
      if (isinstance(s.value, LoraLinear)):
        s.value_proj.lora = LoraModule(s.value_proj.linear.in_features, s.query.linear.out_features, rank, alpha)
      else:
        s.value_proj = LoraLinear(s.value_proj, rank, alpha)

      # Store the lora layers for later
      modules.append({'query': s.query_proj.lora, 'value': s.value_proj.lora})

  # Set parameters in A and B as trainable
  for name, param in model.named_parameters():
    if 'A' in name or 'B' in name:
      param.requires_grad = True

  return modules

# def update_lora_layers(modules, model):
#   # Replace dense layers of the model with LoRA
#   if "roberta" in model.__class__.__name__.lower():
#     for i, layer in enumerate(model.roberta.encoder.layer):
#       layer.attention.self.query = modules[i]['query']
#       layer.attention.self.value = modules[i]['value']

#   elif "deberta" in model.__class__.__name__.lower():
#     for i, layer in enumerate(model.deberta.encoder.layer):
#       layer.attention.self.query = modules[i]['query']
#       layer.attention.self.value = modules[i]['value']

Helper function for checking the total number of parameter and the total number of trainable parameters for a given model.

In [ ]:
def get_trainable_parameters(model):
  trainable_parameters = 0
  parameters = 0
  for param in model.parameters():
    # count the total number of parameters
    parameters += param.numel()
    if param.requires_grad:
      # count the total number of trainable parameters
      trainable_parameters += param.numel()
  return parameters, trainable_parameters

Define hyperparameters specified in the LoRA paper.

In [ ]:
roberta_base_hyperparameters = {
    "mnli": {
      "lr-schedule": "linear",
      "warmup-ratio": 0.06,
      "batch-size": 16,
      "epochs": 30,
      "learning-rate": 5e-04,
      "rank": 8,
      "alpha": 8,
      "max-seq-len": 512
    },
    "sst2": {
      "lr-schedule": "linear",
      "warmup-ratio": 0.06,
      "batch-size": 16,
      "epochs": 60,
      "learning-rate": 5e-04,
      "rank": 8,
      "alpha": 8,
      "max-seq-len": 512
    },
    "mrpc": {
      "lr-schedule": "linear",
      "warmup-ratio": 0.06,
      "batch-size": 16,
      "epochs": 30,
      "learning-rate": 4e-04,
      "rank": 8,
      "alpha": 8,
      "max-seq-len": 512,
    },
    "cola": {
      "lr-schedule": "linear",
      "warmup-ratio": 0.06,
      "batch-size": 32,
      "epochs": 80,
      "learning-rate": 4e-04,
      "rank": 8,
      "alpha": 8,
      "max-seq-len": 512
    },
    "qnli": {
      "lr-schedule": "linear",
      "warmup-ratio": 0.06,
      "batch-size": 32,
      "epochs": 25,
      "learning-rate": 4e-04,
      "rank": 8,
      "alpha": 8,
      "max-seq-len": 512
    },
    "qqp": {
      "lr-schedule": "linear",
      "warmup-ratio": 0.06,
      "batch-size": 16,
      "epochs": 25,
      "learning-rate": 4e-04,
      "rank": 8,
      "alpha": 8,
      "max-seq-len": 512
    },
    "rte": {
      "lr-schedule": "linear",
      "warmup-ratio": 0.06,
      "batch-size": 32,
      "epochs": 80,
      "learning-rate": 5e-04,
      "rank": 8,
      "alpha": 8,
      "max-seq-len": 512
    },
    "stsb": {
      "lr-schedule": "linear",
      "warmup-ratio": 0.06,
      "batch-size": 16,
      "epochs": 40,
      "learning-rate": 4e-04,
      "rank": 8,
      "alpha": 8,
      "max-seq-len": 512
    }
}

roberta_large_hyperparameters = {
    "mnli": {
      "lr-schedule": "linear",
      "warmup-ratio": 0.06,
      "batch-size": 4,
      "epochs": 10,
      "learning-rate": 3e-04,
      "rank": 8,
      "alpha": 16,
      "max-seq-len": 128
    },
    "sst2": {
      "lr-schedule": "linear",
      "warmup-ratio": 0.06,
      "batch-size": 4,
      "epochs": 10,
      "learning-rate": 4e-04,
      "rank": 8,
      "alpha": 16,
      "max-seq-len": 128
    },
    "mrpc": {
      "lr-schedule": "linear",
      "warmup-ratio": 0.06,
      "batch-size": 4,
      "epochs":20,
      "learning-rate": 3e-04,
      "rank": 8,
      "alpha": 16,
      "max-seq-len": 512
    },
    "cola": {
      "lr-schedule": "linear",
      "warmup-ratio": 0.06,
      "batch-size": 4,
      "epochs": 20,
      "learning-rate": 2e-04,
      "rank": 8,
      "alpha": 16,
      "max-seq-len": 128
    },
    "qnli": {
      "lr-schedule": "linear",
      "warmup-ratio": 0.06,
      "batch-size": 4,
      "epochs": 10,
      "learning-rate": 2e-04,
      "rank": 8,
      "alpha": 16,
      "max-seq-len": 128
    },
    "qqp": {
      "lr-schedule": "linear",
      "warmup-ratio": 0.06,
      "batch-size": 4,
      "epochs": 20,
      "learning-rate": 3e-04,
      "rank": 8,
      "alpha": 16,
      "max-seq-len": 512
    },
    "rte": {
      "lr-schedule": "linear",
      "warmup-ratio": 0.06,
      "batch-size": 8,
      "epochs": 20,
      "learning-rate": 4e-04,
      "rank": 8,
      "alpha": 16,
      "max-seq-len": 512
    },
    "stsb": {
      "lr-schedule": "linear",
      "warmup-ratio": 0.06,
      "batch-size": 8,
      "epochs": 30,
      "learning-rate": 2e-04,
      "rank": 8,
      "alpha": 16,
      "max-seq-len": 512
    }
}

deberta_hyperparameters = {
    "mnli": {
      "lr-schedule": "linear",
      "warmup-ratio": 0.1,
      "batch-size": 8,
      "epochs": 5,
      "learning-rate": 1e-04,
      "weight-decay": 0,
      "cls-dropout": 0.15,
      "rank": 8,
      "alpha": 8,
      "max-seq-len": 256
    },
    "sst2": {
      "lr-schedule": "linear",
      "warmup-ratio": 0.1,
      "batch-size": 8,
      "epochs": 16,
      "learning-rate": 6e-05,
      "weight-decay": 0.01,
      "cls-dropout": 0,
      "rank": 8,
      "alpha": 8,
      "max-seq-len": 128
    },
    "mrpc": {
      "lr-schedule": "linear",
      "warmup-ratio": 0.1,
      "batch-size": 32,
      "epochs": 30,
      "learning-rate": 2e-04,
      "weight-decay": 0.01,
      "cls-dropout": 0,
      "rank": 8,
      "alpha": 8,
      "max-seq-len": 128
    },
    "cola": {
      "lr-schedule": "linear",
      "warmup-ratio": 0.1,
      "batch-size": 4,
      "epochs": 10,
      "learning-rate": 1e-04,
      "weight-decay": 0,
      "cls-dropout": 0.1,
      "rank": 8,
      "alpha": 8,
      "max-seq-len": 256
    },
    "qnli": {
      "lr-schedule": "linear",
      "warmup-ratio": 0.1,
      "batch-size": 6,
      "epochs": 8,
      "learning-rate": 1e-04,
      "weight-decay": 0.01,
      "cls-dropout": 0.1,
      "rank": 8,
      "alpha": 8,
      "max-seq-len": 512
    },
    "qqp": {
      "lr-schedule": "linear",
      "warmup-ratio": 0.1,
      "batch-size": 8,
      "epochs": 11,
      "learning-rate": 1e-04,
      "weight-decay": 0.01,
      "cls-dropout": 0.2,
      "rank": 8,
      "alpha": 8,
      "max-seq-len": 320
    },
    "rte": {
      "lr-schedule": "linear",
      "warmup-ratio": 0.1,
      "batch-size": 4,
      "epochs": 11,
      "learning-rate": 2e-04,
      "weight-decay": 0.01,
      "cls-dropout": 0.2,
      "rank": 8,
      "alpha": 8,
      "max-seq-len": 320
    },
    "stsb": {
      "lr-schedule": "linear",
      "warmup-ratio": 0.1,
      "batch-size": 4,
      "epochs": 10,
      "learning-rate": 2e-04,
      "weight-decay": 0.1,
      "cls-dropout": 0.2,
      "rank": 8,
      "alpha": 8,
      "max-seq-len": 128
    }
}

Instantiate models.

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Create roberta-base model
roberta_base = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=2).to(device)
roberta_base_tokenizer = AutoTokenizer.from_pretrained("roberta-base")

# Create roberta-large model
roberta_large = AutoModelForSequenceClassification.from_pretrained("roberta-large", num_labels=2).to(device)
roberta_large_tokenizer = AutoTokenizer.from_pretrained("roberta-large")

# Freeze parameters
for param in roberta_base.parameters():
  param.requires_grad = False
for param in roberta_large.parameters():
  param.requires_grad = False

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

...

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Initialize and check LoRA layers.

In [ ]:
  # Initialize LoRA layers in model
roberta_base_mrpc_modules = init_lora_layers(roberta_base_hyperparameters["mrpc"]["rank"], roberta_base_hyperparameters["mrpc"]["alpha"], roberta_base)
roberta_base_cola_modules = init_lora_layers(roberta_base_hyperparameters["cola"]["rank"], roberta_base_hyperparameters["cola"]["alpha"], roberta_base)
roberta_large_mrpc_modules = init_lora_layers(roberta_large_hyperparameters["mrpc"]["rank"], roberta_large_hyperparameters["mrpc"]["alpha"], roberta_large)
roberta_large_cola_modules = init_lora_layers(roberta_large_hyperparameters["cola"]["rank"], roberta_large_hyperparameters["cola"]["alpha"], roberta_large)

In [ ]:
print(roberta_base)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): LoraLinear(
                (linear): Linear(in_features=768, out_features=768, bias=True)
                (lora): LoraModule()
              )
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): LoraLinear(
                (linear): Linear(in_features=768, out_features=768, bias=True)
                (lora): LoraModule()
              )
              (dr

In [ ]:
print(roberta_large)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): LoraLinear(
                (linear): Linear(in_features=1024, out_features=1024, bias=True)
                (lora): LoraModule()
              )
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): LoraLinear(
                (linear): Linear(in_features=1024, out_features=1024, bias=True)
                (lora): LoraModule()
              )
       

Implement a train function using the trainer api.

In [ ]:
from transformers import DataCollatorWithPadding
from datasets import load_dataset
import evaluate
from transformers import Trainer, TrainingArguments

task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp" : ("question1", "question2"),
    "rte" : ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}

def train_new(name, model, tokenizer, task, hyperparameters):
  h = hyperparameters[task]

  # Load dataset
  raw_datasets = load_dataset("glue", task)

  def tokenize_function(example):
    col1 = task_to_keys[task][0]
    col2 = task_to_keys[task][1]
    if col2 == None:
      return tokenizer(example[col1], max_length=h["max-seq-len"], truncation=True)
    else:
      return tokenizer(example[col1], example[col2], max_length=h["max-seq-len"], truncation=True)

  tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
  data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

  def compute_metrics(eval_pred):
    metric = evaluate.load("glue", task)
    predictions, labels = eval_pred
    if task == "stsb":
      predictions = predictions[:,0]
    else:
      predictions = np.argmax(predictions, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

  metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"

  training_args = TrainingArguments(
      f"{name}-finetuned-{task}", # output_dir
      warmup_ratio=h["warmup-ratio"],
      lr_scheduler_type=h["lr-schedule"],
      per_device_train_batch_size=h["batch-size"],
      per_device_eval_batch_size=h["batch-size"],
      eval_strategy="epoch",
      save_strategy="epoch",
      num_train_epochs=h["epochs"],
      learning_rate=h["learning-rate"],
      load_best_model_at_end=True,
      metric_for_best_model=metric_name,
      report_to="none"
      )

  # Handle mnli-mm and mnli expected keys for eval_dataset
  validation_key = "validation_mismatched" if task=="mnli-mm" else \
                "validation_matched" if task=="mnli" else "validation"

  # Initialize LoRA layers in model
  modules = init_lora_layers(h["rank"], h["alpha"], model)

  # Check trainable parameters
  num_params, num_trainable_params = get_trainable_parameters(model)
  print(f"Parameters={num_params}; Trainable Parameters={num_trainable_params}")

  trainer = Trainer(
      model,
      training_args,
      train_dataset=tokenized_datasets["train"],
      eval_dataset=tokenized_datasets[validation_key],
      data_collator=data_collator,
      tokenizer=tokenizer,
      compute_metrics=compute_metrics
  )
  trainer.train()

  return modules

Try mrpc 5 times with roberta-base.

In [ ]:
roberta_base_mrpc_modules = train_new("roberta-base", roberta_base, roberta_base_tokenizer, "mrpc", roberta_base_hyperparameters)

README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

Parameters=124942082; Trainable Parameters=294912


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.524033,0.801471,0.866116
2,No log,0.461570,0.803922,0.856631
3,0.582000,0.552972,0.732843,0.789981
4,0.582000,0.528302,0.779412,0.850498
5,0.570400,0.623963,0.683824,0.812227
6,0.570400,0.624058,0.683824,0.812227
7,0.636900,0.624440,0.683824,0.812227
8,0.636900,0.624908,0.683824,0.812227
9,0.634200,0.624620,0.683824,0.812227
10,0.634200,0.624428,0.683824,0.812227


In [ ]:
roberta_base_mrpc_modules = train_new("roberta-base", roberta_base, roberta_base_tokenizer, "mrpc", roberta_base_hyperparameters)

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

Parameters=124942082; Trainable Parameters=294912


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.846047,0.683824,0.812227
2,No log,0.663934,0.686275,0.813411
3,0.603400,0.627573,0.683824,0.812227
4,0.603400,0.626920,0.683824,0.812227
5,0.635000,0.624030,0.683824,0.812227
6,0.635000,0.624043,0.683824,0.812227
7,0.636500,0.623332,0.683824,0.811679
8,0.636500,0.624612,0.683824,0.812227
9,0.635300,0.624123,0.683824,0.812227
10,0.635300,0.624572,0.683824,0.812227


In [ ]:
roberta_base_mrpc_modules = train_new("roberta-base", roberta_base, roberta_base_tokenizer, "mrpc", roberta_base_hyperparameters)

Parameters=124942082; Trainable Parameters=294912


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.846047,0.683824,0.812227
2,No log,0.663934,0.686275,0.813411
3,0.603400,0.627573,0.683824,0.812227
4,0.603400,0.626920,0.683824,0.812227
5,0.635000,0.624030,0.683824,0.812227
6,0.635000,0.624043,0.683824,0.812227
7,0.636500,0.623332,0.683824,0.811679
8,0.636500,0.624612,0.683824,0.812227
9,0.635300,0.624123,0.683824,0.812227
10,0.635300,0.624572,0.683824,0.812227


In [ ]:
roberta_base_mrpc_modules = train_new("roberta-base", roberta_base, roberta_base_tokenizer, "mrpc", roberta_base_hyperparameters)

Parameters=124942082; Trainable Parameters=294912


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.846047,0.683824,0.812227
2,No log,0.663934,0.686275,0.813411
3,0.603400,0.627573,0.683824,0.812227
4,0.603400,0.626920,0.683824,0.812227
5,0.635000,0.624030,0.683824,0.812227
6,0.635000,0.624043,0.683824,0.812227
7,0.636500,0.623332,0.683824,0.811679
8,0.636500,0.624612,0.683824,0.812227
9,0.635300,0.624123,0.683824,0.812227
10,0.635300,0.624572,0.683824,0.812227


In [ ]:
roberta_base_mrpc_modules = train_new("roberta-base", roberta_base, roberta_base_tokenizer, "mrpc", roberta_base_hyperparameters)

Parameters=124942082; Trainable Parameters=294912


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.846047,0.683824,0.812227
2,No log,0.663934,0.686275,0.813411
3,0.603400,0.627573,0.683824,0.812227
4,0.603400,0.626920,0.683824,0.812227
5,0.635000,0.624030,0.683824,0.812227
6,0.635000,0.624043,0.683824,0.812227
7,0.636500,0.623332,0.683824,0.811679
8,0.636500,0.624612,0.683824,0.812227
9,0.635300,0.624123,0.683824,0.812227
10,0.635300,0.624572,0.683824,0.812227


Try cola 5 times with roberta-base.

In [ ]:
roberta_base_cola_modules = train_new("roberta-base", roberta_base, roberta_base_tokenizer, "cola", roberta_base_hyperparameters)

train-00000-of-00001.parquet:   0%|          | 0.00/251k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/37.6k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/37.7k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8551 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1043 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/8551 [00:00<?, ? examples/s]

Map:   0%|          | 0/1043 [00:00<?, ? examples/s]

Map:   0%|          | 0/1063 [00:00<?, ? examples/s]

Parameters=124942082; Trainable Parameters=294912


Epoch,Training Loss,Validation Loss,Matthews Correlation
1,No log,0.666997,0.000000
2,0.549500,0.522214,0.431194
3,0.549500,0.627620,0.383445
4,0.492400,0.630741,0.130696
5,0.492400,0.565120,0.281600
6,0.516400,0.514775,0.433697
7,0.516400,0.515413,0.348979
8,0.521600,0.585113,0.215003
9,0.521600,0.501122,0.400359
10,0.511200,0.609906,0.318550


In [ ]:
roberta_base_cola_modules = train_new("roberta-base", roberta_base, roberta_base_tokenizer, "cola", roberta_base_hyperparameters)

Parameters=124942082; Trainable Parameters=294912


Epoch,Training Loss,Validation Loss,Matthews Correlation
1,No log,0.576489,0.092846
2,0.550900,0.492374,0.435565
3,0.550900,0.581063,0.385419
4,0.496200,0.599266,0.186010
5,0.496200,0.522402,0.419078
6,0.525900,0.520523,0.376242
7,0.525900,0.563538,0.239516
8,0.543900,0.544842,0.232337
9,0.543900,0.621855,0.290033
10,0.543000,0.638441,0.314777


In [ ]:
roberta_base_cola_modules = train_new("roberta-base", roberta_base, roberta_base_tokenizer, "cola", roberta_base_hyperparameters)

Parameters=124942082; Trainable Parameters=294912


Epoch,Training Loss,Validation Loss,Matthews Correlation
1,No log,0.576489,0.092846
2,0.550900,0.492374,0.435565
3,0.550900,0.581063,0.385419
4,0.496200,0.599266,0.186010
5,0.496200,0.522402,0.419078
6,0.525900,0.520523,0.376242
7,0.525900,0.563538,0.239516
8,0.543900,0.544842,0.232337
9,0.543900,0.621855,0.290033
10,0.543000,0.638441,0.314777


In [ ]:
roberta_base_cola_modules = train_new("roberta-base", roberta_base, roberta_base_tokenizer, "cola", roberta_base_hyperparameters)

Parameters=124942082; Trainable Parameters=294912


Epoch,Training Loss,Validation Loss,Matthews Correlation
1,No log,0.576489,0.092846
2,0.550900,0.492374,0.435565
3,0.550900,0.581063,0.385419
4,0.496200,0.599266,0.186010
5,0.496200,0.522402,0.419078
6,0.525900,0.520523,0.376242
7,0.525900,0.563538,0.239516
8,0.543900,0.544842,0.232337
9,0.543900,0.621855,0.290033
10,0.543000,0.638441,0.314777


In [ ]:
roberta_base_cola_modules = train_new("roberta-base", roberta_base, roberta_base_tokenizer, "cola", roberta_base_hyperparameters)

Parameters=124942082; Trainable Parameters=294912


Epoch,Training Loss,Validation Loss,Matthews Correlation
1,No log,0.576489,0.092846
2,0.550900,0.492374,0.435565
3,0.550900,0.581063,0.385419
4,0.496200,0.599266,0.186010
5,0.496200,0.522402,0.419078
6,0.525900,0.520523,0.376242
7,0.525900,0.563538,0.239516
8,0.543900,0.544842,0.232337
9,0.543900,0.621855,0.290033
10,0.543000,0.638441,0.314777


Try mrpc with roberta-large

In [ ]:
roberta_large_mrpc_modules = train_new("roberta-large", roberta_large, roberta_large_tokenizer, "mrpc", roberta_large_hyperparameters)

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

Parameters=356148226; Trainable Parameters=786432


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.680200,0.626200,0.683824,0.812227
2,0.651000,0.627262,0.683824,0.812227
3,0.643400,0.624526,0.683824,0.812227
4,0.642900,0.624481,0.683824,0.812227
5,0.648400,0.624664,0.683824,0.812227
6,0.631900,0.638514,0.683824,0.812227
7,0.637400,0.626591,0.683824,0.812227
8,0.650800,0.632081,0.683824,0.812227
9,0.638000,0.631823,0.683824,0.812227
10,0.649400,0.625250,0.683824,0.812227


Try cola with roberta-large

In [ ]:
roberta_large_cola_modules = train_new("roberta-large", roberta_large, roberta_large_tokenizer, "cola", roberta_large_hyperparameters)

README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/251k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/37.6k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/37.7k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8551 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/1043 [00:00<?, ? examples/s]

Map:   0%|          | 0/1063 [00:00<?, ? examples/s]

Parameters=356148226; Trainable Parameters=786432


Epoch,Training Loss,Validation Loss,Matthews Correlation
1,0.625200,0.618848,0.000000


KeyboardInterrupt: 